In [1]:
import serial
from serial.tools import list_ports
from time import sleep


In [2]:
class communicator():
    
    def __init__(self, verbose: bool=True):
        self.port = self.find_device(verbose)
        self.s = serial.Serial(self.port,115200,timeout=5)# establish connection with the device and open the device
        self.verbose = verbose
        self.com_setup()
        
    def find_device(self, verbose: bool) -> str:
        ports = list_ports.comports()
        #For Debugging
        #print("Available Serial Ports:")
        #for p in ports:
            #print(f"Port:{p.device} | Description:{p.description} | HWID:{p.hwid}")
            
        possible_devices = list()
        i=0
        
        if verbose: # for optional debugging
            print("Available COMs:")
            
        for p in ports:
            if "CP210x" in p.description: # Checking a device with CP210x in description is connected to the com_port
                possible_devices.append(p.device)
                if verbose:
                    print("%i >" %i, end='')
                i+=1
                
            if verbose: print("\t%-5s: %-30s [%s]" %(p.device, p.description, p.hwid))
        # For returning the device name     
        if not verbose and len(possible_devices) !=1:
            return None
        elif len(possible_devices) ==0:
            print("NO USABLE DEVICE FOUND")
            return None
        elif len(possible_devices)>1:
            print("Type Nr. to Use")
            return possible_devices[int(input())]
        else:
            return possible_devices[0]
        
    def com_setup(self):
        print_str = str()
        
        self.s.reset_input_buffer() # clears all the data waiting in the input buffer
        self.s.reset_output_buffer() #clears all the data waiting in the output buffer
        
        self.s.write(b"restart\n") #RESTARTS THE dEVICE THIS IS an ESP 32 feature: ESP.restart()
        sleep(1)
        
        print_str += self.s.read_all().decode(errors="ignore") # read_all() Reads all available bytes currently waiting in the serial input buffer.
                                                # .decode() Converts the byte read from the serial port into a string
        print(print_str)#
        self.s.write(b"print_mode -m RAW\n")
        sleep(0.5)
        print_str += self.s.read_all().decode(errors='ignore')
        
        if self.verbose:
            print(print_str)
        
        

In [3]:
comm = communicator() # instance for running the communicator()

Available COMs:
0 >	COM4 : Silicon Labs CP210x USB to UART Bridge (COM4) [USB VID:PID=10C4:EA60 SER=0001 LOCATION=1-3]
ets Jul 29 2019 12:21:46

rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid headets Jul 29 2019 12:21:46

rst:0x10 (RTCWDT_RTC_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid heaets Jul 29 2019 12:21:46

rst:0x10 (RTCWDT_RTC_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff

ets Jul 29 2019 12:21:46

rst:0x1 (POWERON_RESET),boot:0x13 (SP